In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
## Imports and environment variables 
import torch, wandb, os, pandas as pd 
from travis_attack.utils import set_seed, set_session_options, setup_logging, setup_parser, resume_wandb_run, display_all, print_important_cfg_vars
from travis_attack.config import Config
from travis_attack.models import prepare_models, get_optimizer
from travis_attack.data import ProcessedDataset
from travis_attack.trainer import Trainer
from travis_attack.insights import (postprocess_df, create_and_log_wandb_postrun_plots, get_training_dfs)
from fastcore.basics import in_jupyter

import logging 
logger = logging.getLogger("run")

import warnings
warnings.filterwarnings("ignore", message="Passing `max_length` to BeamSearchScorer is deprecated")  # works anyway for diverse beam search 

In [ ]:
from nbdev.export import notebook2script
notebook2script()

!jupyter nbconvert \
    --TagRemovePreprocessor.enabled=True \
    --TagRemovePreprocessor.remove_cell_tags="['hide']" \
    --TemplateExporter.exclude_markdown=True \
    --to python "run.ipynb"

Converted 00_utils.ipynb.
Converted 02_tests.ipynb.
Converted 03_config.ipynb.
Converted 07_models.ipynb.
Converted 10_data.ipynb.
Converted 20_trainer.ipynb.
Converted 25_insights.ipynb.
Converted baselines.ipynb.
Converted baselines_analysis.ipynb.
Converted index.ipynb.
Converted pp_eval_baselines.ipynb.
Converted run.ipynb.
Converted show_examples.ipynb.
Converted test_pp_model.ipynb.
[NbConvertApp] Converting notebook run.ipynb to python


In [ ]:
cfg = Config()  # default values
if not in_jupyter():  # override with any -- options when running with command line
    parser = setup_parser()
    newargs = vars(parser.parse_args())
    for k,v in newargs.items(): 
        if v is not None: 
            if k in cfg.pp.keys():  cfg.pp[k] = v
            else:                   setattr(cfg, k, v)
if cfg.use_small_ds:  cfg = cfg.small_ds()
set_seed(cfg.seed)
set_session_options()
setup_logging(cfg, disable_other_loggers=True)
vm_tokenizer, vm_model, pp_tokenizer, pp_model, ref_pp_model, sts_model, nli_tokenizer, nli_model, cfg = prepare_models(cfg)
optimizer = get_optimizer(cfg, pp_model)
ds = ProcessedDataset(cfg, vm_tokenizer, vm_model, pp_tokenizer, sts_model, load_processed_from_file=False)


travis_attack.data: INFO     Will load dataset rotten_tomatoes with use_small_ds set to True
travis_attack.data: INFO     Will load dataset rotten_tomatoes with use_small_ds set to True


In [ ]:
cfg.wandb['mode'] = 'online'
trainer = Trainer(cfg, vm_tokenizer, vm_model, pp_tokenizer, pp_model, ref_pp_model, sts_model, nli_tokenizer, nli_model, optimizer,
         ds, initial_eval=True)

#print_important_cfg_vars(cfg)
trainer.train()

wandb: wandb version 0.12.16 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


travis_attack.trainer: INFO     Launching initial eval run: train
travis_attack.trainer: INFO     Launching initial eval run: train


travis_attack.trainer: INFO     Launching initial eval run: valid
travis_attack.trainer: INFO     Launching initial eval run: valid


travis_attack.trainer: INFO     Launching initial eval run: test
travis_attack.trainer: INFO     Launching initial eval run: test


travis_attack.trainer: INFO     Now on epoch 1 of 2
travis_attack.trainer: INFO     Now on epoch 1 of 2


travis_attack.trainer: INFO     Now on epoch 2 of 2
travis_attack.trainer: INFO     Now on epoch 2 of 2


In [ ]:
#df = pd.read_csv(cfg.path_results + "run_results.csv")
#display_all(df)

In [ ]:
# df = pd.read_csv(f'{cfg.path_run}training_step.csv')
# #display_all(df.query('idx==1'))
# df.columns

In [ ]:
#trainer.run.finish()

In [ ]:
# # ## TO RESUME RUN
# cfg = Config()
# cfg.run_id = 'b9r88lmz'
# cfg.run_name = "fresh-cosmos-151"
# cfg.path_run = f"{cfg.path_checkpoints}{cfg.run_name}/"
# run = resume_wandb_run(cfg)


In [ ]:
df_d = get_training_dfs(cfg.path_run, postprocessed=False)
for k, df in df_d.items(): 
    df_d[k] = postprocess_df(df, filter_idx=None, num_proc=1)
    df_d[k].to_pickle(f"{cfg.path_run}{k}_postprocessed.pkl")    
create_and_log_wandb_postrun_plots(df_d)
trainer.run.finish()
#run.finish()

travis_attack.insights: INFO     Dataframes have shapes ['training_step: (58, 49)', 'train: (696, 20)', 'valid: (144, 20)', 'test: (32, 20)']
travis_attack.insights: INFO     Dataframes have shapes ['training_step: (58, 49)', 'train: (696, 20)', 'valid: (144, 20)', 'test: (32, 20)']
/data/tproth/travis_attack/travis_attack/insights.py:69: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('pp_changed', 1) # don't need this anymore
travis_attack.insights: INFO     Adding text metrics for column orig
travis_attack.insights: INFO     Adding text metrics for column orig


travis_attack.insights: INFO     Adding text metrics for column pp
travis_attack.insights: INFO     Adding text metrics for column pp


travis_attack.insights: INFO     Calculating metric differences between orig and pp
travis_attack.insights: INFO     Calculating metric differences between orig and pp
travis_attack.insights: INFO     Calculating text pair statistics for (orig, pp) unique pairs
travis_attack.insights: INFO     Calculating text pair statistics for (orig, pp) unique pairs


/data/tproth/travis_attack/travis_attack/insights.py:69: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('pp_changed', 1) # don't need this anymore
travis_attack.insights: INFO     Adding text metrics for column orig
travis_attack.insights: INFO     Adding text metrics for column orig


travis_attack.insights: INFO     Adding text metrics for column pp
travis_attack.insights: INFO     Adding text metrics for column pp


...

travis_attack.insights: INFO     Calculating metric differences between orig and pp
travis_attack.insights: INFO     Calculating metric differences between orig and pp
travis_attack.insights: INFO     Calculating text pair statistics for (orig, pp) unique pairs
travis_attack.insights: INFO     Calculating text pair statistics for (orig, pp) unique pairs


/data/tproth/travis_attack/travis_attack/insights.py:69: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('pp_changed', 1) # don't need this anymore
travis_attack.insights: INFO     Adding text metrics for column orig
travis_attack.insights: INFO     Adding text metrics for column orig


travis_attack.insights: INFO     Adding text metrics for column pp
travis_attack.insights: INFO     Adding text metrics for column pp


travis_attack.insights: INFO     Calculating metric differences between orig and pp
travis_attack.insights: INFO     Calculating metric differences between orig and pp
travis_attack.insights: INFO     Calculating text pair statistics for (orig, pp) unique pairs
travis_attack.insights: INFO     Calculating text pair statistics for (orig, pp) unique pairs


%_of_tokens_above_prob_1/vocab_size,█████▁██
%_of_tokens_above_prob_1e-05,█████▁██
%_of_tokens_above_prob_1e-06,█████▁██
%_of_tokens_above_prob_1e-07,█████▁██
%_of_tokens_above_prob_1e-08,█████▁██
%_of_tokens_above_prob_1e-09,█████▁██
acc_batch_n_examples,██▁▁██▁▁
acc_num,▁█▁█▁█▁█
any_adv_example_proportion-test,▁▁
any_adv_example_proportion-train,▁▁▁
any_adv_example_proportion-valid,▁▁▁
